In [4]:
# Install dependencies
!pip3 install paramiko scp

     |████████████████████████████████| 206 kB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 168 kB/s  eta 0:00:01
     |████████████████████████████████| 961 kB 44.5 MB/s eta 0:00:01


In [27]:
!pip install avro-python3

  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=f16a95cc6652ac6f8c06d5fa3548e4a4ba6a5f0b7012dd1e2750bdf571f88db3
  Stored in directory: /home/jovyan/.cache/pip/wheels/bb/73/e9/d273421f5723c4bf544dcf9eb097bda94421ef8d3252699f0a
Successfully built avro-python3


In [30]:
!pip install python-snappy

     |████████████████████████████████| 54 kB 977 kB/s  eta 0:00:01


In [1]:
# This notebook is for indexing the files in SFTP server.
# It indexes by user id and time of the records

In [ ]:
# import libraries

In [21]:
from sftp_client import RemoteClient

In [ ]:
# Create SFTP client

In [22]:
host = 'radar-base-nas.brc.iop.kcl.ac.uk'
user = 'yranjan'
ssh_key_filepath = 'truenas'
remote_path = '/mnt/pool0/covid_collab/topics'

local_file_directory = 'data'

client = RemoteClient(host, user, ssh_key_filepath, remote_path)

In [25]:
client._connect()

In [26]:
client.download_file(remote_path)

AttributeError: 'RemoteClient' object has no attribute 'connect'

In [ ]:
# Read existing file names and hashes to compare and only update files which have changed hash values (similar to rsync)

In [1]:
import copy
import json
import avro
from avro.datafile import DataFileWriter, DataFileReader
from avro.io import DatumWriter, DatumReader
import asyncio

In [ ]:
# Read data from an avro file
with open('connect_fitbit_intraday_heart_rate+0+1509748614+1509759980.avro', 'rb') as f:
    reader = DataFileReader(f, DatumReader())
    metadata = copy.deepcopy(reader.meta)
    schema_from_file = json.loads(metadata['avro.schema'])
    datum = [data for data in reader]
    reader.close()

asyncio.create_task(process_data(datum, schema))

In [ ]:
# Process tasks with coroutines

list_files = []

for fi in list_files:
    with open(fi, 'rb') as f:
    reader = DataFileReader(f, DatumReader())
    metadata = copy.deepcopy(reader.meta)
    schema_from_file = json.loads(metadata['avro.schema'])
    datum = [data for data in reader]
    reader.close()
    
    asyncio.create_task(process_data(datum, schema))

In [10]:
import datetime
import os
from pathlib import Path

In [ ]:
time_column = 'time'
data_extract_path = 'data/avro/'

async def process_data(data, schema):
    """
    This restructures the data into the format /{projectId}/{userId}/{date}.avro
    So groups by projectId and userId and creates a single avro for each day.
    """
    for data in datum:
        projectId = data['key']['projectId']
        userId = data['key']['userId']
        data_path = os.path.join(data_extract_path, projectId, userId)
        try:
            os.makedirs(data_path)
        except FileExistsError:
            pass
        time = datetime.datetime.fromtimestamp(data['value'][time_column])
        my_file = Path(os.path.join(data_path, time.date(), '.avro'))
        my_file_lock = Path(os.path.join(data_path, time.date(), '.lock'))
        
        while my_file_lock.exists():
            # while another thread is processing this file, we do not write to it
            await time.sleep(0.1)
        try:
            my_abs_path = my_file.resolve(strict=True)
        except FileNotFoundError:
            # doesn't exist
            # open with write mode
            
            # TODO: acquire lock
            
            await with open(my_file, 'wb') as f:
                writer = DataFileWriter(f, DatumWriter(), schema)
                writer.append(data)
                writer.close()
        else:
            # exists
            # open in append mode
            
            # TODO: acquire lock
            
            await with open(my_file, 'ab') as f:
                writer = DataFileWriter(f, DatumWriter(), schema)
                writer.append(data)
                writer.close()
        finally:
            # TODO: delete lock file
                
                

In [5]:
display(users[0])

{'key': {'projectId': 'RADAR-AIIMS-2-TRIALS-KCL-s1',
  'userId': 'cdc4bf33-c76c-4a51-b9ee-f326c5b50846',
  'sourceId': 'a226a6d5-f208-457c-a38a-17ea9a0593f8'},
 'value': {'time': 1619433468.0,
  'timeReceived': 1619520049.776,
  'timeInterval': 1,
  'heartRate': 75}}

In [14]:
print(datetime.datetime.fromtimestamp(1619433468.0).date())

2021-04-26


In [ ]:
# read each avro file and index according to userId and time

In [ ]:
# Use couroutines and tasks to make this process faster

In [ ]:
# check hash